In [127]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from decimal import Decimal, ROUND_HALF_UP

def total_excessive_delay(df_ted):
    df_ted['TED'] = round(df_ted['ED'] * df_ted['PK_HR'], 2)
    df_ted['TED'] = round(df_ted['TED']) # Need this line to fix excel rounding weirdness

    #Groupby for summation by TMC
    #ted_operations = ({'TED' : 'sum'})
    #df_ted = df_ted.groupby('tmc_code', as_index=False).agg(ted_operations)
    return df_ted
    
def peak_hr(df_pk):
    df_pk['PK_HR'] = round(df_pk['DirAADT_AUTO']/4)
    return df_pk
    
def excessive_delay(df_ed):
    df_ed['ED'] = df_ed['RSD']/3600 # check this value hundredths of an hour
    #df_ed['ED'] = df_ed['ED'].round(3)
    df_ed['ED'] = df_ed['ED'].apply(Decimal(df_ed['ED']).quantize('.001'), rounding=ROUND_HALF_UP)
    df_ed['ED'] = np.where(df_ed['ED'] >= 0, df_ed['ED'], 0)
    return df_ed

def RSD(df_rsd):
    # returns travel time segment delay calculations.
    df_rsd['RSD'] = df_rsd['SD'] - df_rsd['2017_EDT']
    df_rsd['RSD'] = np.where(df_rsd['RSD'] >= 0, df_rsd['RSD'], 0)
    return df_rsd
        
def segment_delay(df_sd):
    # returns SD spreadsheet value.
    df_sd['SD'] = df_sd['travel_time_seconds'].round()
    return df_sd

def AADT_splits(df_in):
    # apply ODOT methodology AADT_spit calculations to dataframe and create new columns.
    # df_in['aadt_singl] = Dir_AADT_BUD, 
    # df_in['aadt_combi'] = DirAADT_TRK
    df_in['DirAADT_AUTO'] = df_in['aadt'] - (df_in['aadt_singl'] + df_in['aadt_combi'])
    df_in['pct_auto'] = df_in['DirAADT_AUTO']/df_in['aadt']
    df_in['pct_bus'] = df_in['aadt_singl']/df_in['aadt']
    df_in['pct_truck'] = df_in['aadt_combi']/df_in['aadt']
    return df_in

def main():
    pd.set_option('display.max_rows', None)

    df = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/Feb2017_test.csv')) #fix in script implementation
    # Filter by timestamps
    df['measurement_tstamp'] = pd.to_datetime(df['measurement_tstamp'])
    df = df[df['measurement_tstamp'].dt.weekday.isin([0, 1, 2, 3, 4])] # Capture weekdays only
    df = df[df['measurement_tstamp'].dt.hour.isin([6, 7, 8, 9, 16, 17, 18, 19])]
    
    df_meta = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/TMC_Identification.csv'), 
                          usecols=['tmc', 'miles', 'tmclinear', 'aadt', 'aadt_singl', 'aadt_combi' ])
    df_odot = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/odot_edt.csv'))    
    df = pd.merge(df, df_meta, left_on=df['tmc_code'], right_on=['tmc'], how='inner')
    df = pd.merge(df, df_odot, left_on=df['tmc_code'], right_on=['TMC'], how='inner')
    
    df = AADT_splits(df)
    df = segment_delay(df)
    df = RSD(df)
    df = excessive_delay(df)
    df = peak_hr(df)
    df = total_excessive_delay(df)
    #print(df)                                       
    # Return a selected row for test purposes only
    df = df[['RSD','PK_HR','ED', 'tmc_code', 'TED']]
    print(df.loc[df['tmc_code'] == '114-04368'])

In [128]:
main()

TypeError: conversion from Series to Decimal is not supported

In [105]:
from decimal import Decimal, ROUND_HALF_UP
num = .0225
print (Decimal(num))
Decimal(num).quantize(Decimal('.001'), rounding=ROUND_HALF_UP)


0.022499999999999999167332731531132594682276248931884765625


Decimal('0.022')